# Synthetic Data Generation

### Synthetic data from real data

합성 데이터 생성은 데이터 요구 사항 증가와 데이터에 대한 프라이버시 문제 등으로 인해 주목받고 있는 분야이다. 은닉 마르코프 모델(HMM)은 합성 데이터를 생성하는 대표적인 모델 중 하나이다. 

합성 데이터는 실제 데이터의 통계적 특성을 모방하는 과정에서 생성된 데이터이다. 데이터는 원 데이터로부터 모델링되어야 한다는 믿음이 있지만, 반드시 실제 데이터에서 합성 데이터를 생성하는 것 만이 유일한 방법은 아니다. 합성 데이터를 생성하는 방법에는 세 가지 방법이 알려져 있다.

1. 실제 데이터를 이용해 합성 데이터를 생성하는 방법. 이 프로세스는 실제 데이터를 가져오는 것에서 시작해 데이터 분포를 모델링하고 마지막으로 해당 분포 모델로부터 합성 데이터를 샘플링한다.

2. 합성 데이터는 모델 또는 지식으로부터 얻을 수 있다. 일반적으로 이러한 유형의 합성 데이터 생성은 기존 모델을 사용하거나 연구자의 지식을 바탕으로 생성된다.

3. 하이브리드 프로세스는 앞의 두 단계를 모두 포함한다. 이런 하이브리드 프로세스는 데이터의 일부만 사용 가능할 때 많이 적용되며, 실제 데이터로 합성 데이터를 일부 생성한 뒤 나머지 부분을 모델로부터 얻어온다.

그렇다면 합성 데이터의 품질은 어떻게 평가할 수 있을까? 일반적으로는 KL-분산, 구별 가능성(Distinguishable), ROC 곡선, 평균&중앙값 4가지 방법론이 사용된다. 

구별 가능성(Distinguishable)은 분류 모델이 실제 데이터와 합성 데이터를 구분하는 경우 실제 데이터에 1을 할당한다. 만일 실제 데이터와 합성 데이터를 구분하지 못한다면 0을 할당한다. 출력이 1에 가까우면 데이터는 실제라고 예측하고, 그렇지 않으면 성향 점수(Propensity score)를 사용해 합성 데이터라고 예측한다.

혹은 실제 데이터와 합성 데이터의 평균과 분산과 같이 주요 통계량을 이용해 합성 데이터가 실제 데이터를 잘 모방하는지 파악할 수 있다.

다음은 실제 데이터를 이용해 합성 데이터를 만들어 내는 코드이다.

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
import matplotlib. pyplot as plt
import yfinance as yf
import datetime
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

In [2]:
X, y = fetch_california_housing(return_X_y=True)

In [3]:
california_housing=np.column_stack([X, y])
california_housing_df=pd.DataFrame(california_housing)
california_housing_df=california_housing_df.iloc[:15000, :]

In [4]:
california_housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  float64
 1   1       15000 non-null  float64
 2   2       15000 non-null  float64
 3   3       15000 non-null  float64
 4   4       15000 non-null  float64
 5   5       15000 non-null  float64
 6   6       15000 non-null  float64
 7   7       15000 non-null  float64
 8   8       15000 non-null  float64
dtypes: float64(9)
memory usage: 1.0 MB


### 생성적 적대 네트워크(GAN)

생성적 적대 네트워크(이하 GAN)는 두가지 모델로 이뤄져 있다. 하나는 데이터를 합성하는 모델이며, 하나는 입력받은 데이터가 합성된 데이터인지, 아니면 실제 데이터인지 판별하는 모델이다. GAN은 데이터를 합성한 뒤 (혹은 그냥 진짜 데이터를) 판별 모델이 받아 실제인지 합성인지 판별하고 이를 다시 합성 모델에 전달하는 식으로 훈련이 진행된다. 현금 위조범과 위조 판별사를 생각해보자. 현금 위조범의 첫 결과물은 너무 조잡하여 위조 판별사는 간단하게 실제 현금과 위조 현금을 구분할 수 있을 것이다. 하지만 이 결과를 위조범이 계속 받게 되어 시간이 지날수록 더욱 정교한 위조 현금이 생성될 것이다. 결과적으로, 위조 판별사는 실제 현금과 위조 현금을 분별하지 못할 것이다.

<center>
    
<img src="./Image/GAN.png" width="700px" height="400px">
    
</center>

이를 수식적으로 서술하면 다음과 같다.

$$ \min_G \max_D V(D, G) = E_{x\sim p_{data}(x)}[logD(x)] + E_{z\sim p_{z}(z)}[log(1-D(G(z)))]$$

여기서 G는 생성 모델을 의미하며, D는 판별 모델을 의미한다. 생성 모델은 데이터 x로부터 생성 분포 $p_g$를 학습하고자 한다. 따라서 생성 모델은 사전 잡음 분포 $p_z(z)$로부터 데이터 스페이스 $G(z;\theta_g)$ 로 매핑하는 함수를 만든다. 여기서 Z는 Latent vector라고도 하며 데이터의 분포를 잘 설명하는 잠재 공간에서의 벡터를 의미한다. 판별 모델 $D(x;\theta_d)$는 x가 생성 분포 $p_g$가 아닌 데이터 x에서 올 확률을 계산한다. 앞쪽 파트 $E_{x\sim p_{data}(x)}[logD(x)]$는 판별 모델로 온 데이터가 실제 데이터에서 왔을 것으로 판단되는 기대값을 의미하며, $E_{z\sim p_{z}(z)}[log(1-D(G(z)))]$는 데이터가 생성 모델로부터 온 것으로 판단하는 기대값을 의미한다. 

분류모델은 해당 손실함수를 최대화해야 하고 생성모델은 손실함수를 최소화 해야 한다. 그러기 위해서는 분류 모델의 입장에서는 $D(x)=1, \ D(G(z))=0$이 되어야 한다. 즉 생성모델이 만들어 낸 데이터는 가짜로 분류해야 하고, 진짜 데이터는 진짜로 판별해야 한다. 생성 모델의 입장에서는 $D(G(z))=1$로 만들어야 한다. 즉, 분류모델이 진짜라고 판단할 수준의 데이터를 만들어야 한다.

### 조건부 GAN(Conditional Generative Adversarial Nets)

조건부 GAN(이하 CGAN)은 GAN과 비슷하나 약간의 차이점을 가진다. 생성 모델과 판별 모델이 새로운 정보 y에 근거하여 만들어져야 한다는 것이다. 여기서 y는 class label, 다른 양식의 데이터 등 어떠한 보조적 데이터든 될 수 있다. 이 사전 정보는 추가적인 input layer로 생성 모델과 판별 모델 모두에게 주어진다.

수식적으로 설명하면 다음과 같다. 생성모델의 사전 input noize(혹은 Latent vector) $p_z(z)$와 y는 joint hidden representation으로 결합된다. 그리고 적대적 훈련 기법은 joint hidden representation을 구현하는 과정에 충분한 유동성을 제공한다.

CGAN의 목적함수는 다음과 같이 변경된다.

$$ \min_G \max_D V(D, G) = E_{x\sim p_{data}(x)}[logD(x\vert y)] + E_{z\sim p_{z}(z)}[log(1-D(G(z\vert y)))]$$

<center>
    
<img src="./Image/CGAN.png" width="700px" height="400px">
    
</center>

`ctgan` 라이브러리는 생성적 적대 네트워크(GAN)모델을 이용해 원본 데이터에 대한 충실도가 높은 합성 데이터를 생성해낸다.

In [5]:
column=[str(i) for i in range(9)]
california_housing_df.columns=column

In [6]:
from ctgan import CTGAN

ctgan = CTGAN(epochs=10)
ctgan.fit(california_housing_df, california_housing_df.columns)
synt_sample = ctgan.sample(len(california_housing_df))

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 13663797636 bytes.

In [ ]:
california_housing_df.describe()

In [ ]:
synt_sample.describe()

In [ ]:
from sdv.evaluation import evaluate

evaluate(synt_sample, california_housing_df)

In [ ]:
from table_evaluator import TableEvaluator

table_evaluator =  TableEvaluator(california_housing_df, synt_sample)

table_evaluator.visual_evaluation()

### Synthetic data from model

In [ ]:
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
X, y = make_regression(n_samples=1000, n_features=3, noise=0.2,
                       random_state=123)

plt.scatter(X[:, 0], X[:, 1], alpha= 0.3, cmap='Greys', c=y)

In [ ]:
plt.figure(figsize=(18, 18))
k = 0

for i in range(0, 10):
    X, y = make_regression(n_samples=100, n_features=3, noise=i,
                           random_state=123) 
    k+=1
    plt.subplot(5, 2, k)
    profit_margin_orange = np.asarray([20, 35, 40])
    plt.scatter(X[:, 0], X[:, 1], alpha=0.3, cmap=cm.Greys, c=y)
    plt.title('Synthetic Data with Different Noises: ' + str(i))
plt.show()

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
plt.figure(figsize=(18, 18))
k = 0

for i in range(2, 6):
    X, y = make_classification(n_samples=100,
                               n_features=4,
                               n_classes=i,
                               n_redundant=0,
                               n_informative=4,
                               random_state=123)
    k+=1
    plt.subplot(2, 2, k)
    plt.scatter(X[: ,0], X[:, 1], alpha=0.8, cmap='gray', c=y)
    plt.title('Synthetic Data with Different Classes: ' + str(i))
plt.show()

## Synthetic Data for Unsupervised Learning

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
X, y = make_blobs(n_samples=100, centers=2, 
                      n_features=2, random_state=0)

In [ ]:
plt.figure(figsize=(18, 18))
k = 0
for i in range(2, 6):
    X, y = make_blobs(n_samples=100, centers=i,
                      n_features=2, random_state=0)
    k += 1
    plt.subplot(2, 2, k)
    my_scatter_plot = plt.scatter(X[:, 0], X[:, 1],
                                  alpha=0.3, cmap='gray', c=y)
    plt.title('Synthetic Data with Different Clusters: ' + str(i))
plt.show()

## HMM

In [ ]:
ff = pd.read_csv('datasets/FF3.csv', skiprows=4)
ff = ff.rename(columns={'Unnamed: 0': 'Date'})
ff = ff.iloc[:-1]
ff.head()

In [ ]:
ff.info()

In [ ]:
ff['Date'] = pd.to_datetime(ff['Date'])
ff.set_index('Date', inplace=True)
ff_trim = ff.loc['2000-01-01':]

In [ ]:
ff_trim.head()

In [ ]:
ticker = 'SPY'
start = datetime.datetime(2000, 1, 3)
end = datetime.datetime(2021, 4, 30)
SP_ETF = yf.download(ticker, start, end, interval='1d').Close

In [ ]:
ff_merge = pd.merge(ff_trim, SP_ETF, how='inner', on='Date')

In [ ]:
SP = pd.DataFrame()
SP['Close']= ff_merge['Close']

In [ ]:
SP['return'] = (SP['Close'] / SP['Close'].shift(1))-1

In [ ]:
from hmmlearn import hmm

In [ ]:
hmm_model = hmm.GaussianHMM(n_components=3,
                            covariance_type="full",
                            n_iter=100)

In [ ]:
hmm_model.fit(np.array(SP['return'].dropna()).reshape(-1, 1))
hmm_predict = hmm_model.predict(np.array(SP['return'].dropna())
                                .reshape(-1, 1))
df_hmm = pd.DataFrame(hmm_predict)

In [ ]:
ret_merged = pd.concat([df_hmm,SP['return'].dropna().reset_index()],
                       axis=1)
ret_merged.drop('Date',axis=1, inplace=True)
ret_merged.rename(columns={0:'states'}, inplace=True)
ret_merged.dropna().head()

In [ ]:
ret_merged['states'].value_counts()

In [ ]:
state_means = []
state_std = []

for i in range(3):
    state_means.append(ret_merged[ret_merged.states == i]['return']
                       .mean())
    state_std.append(ret_merged[ret_merged.states == i]['return']
                     .std())
print('State Means are: {}'.format(state_means))
print('State Standard Deviations are: {}'.format(state_std))

In [ ]:
print(f'HMM means\n {hmm_model.means_}')
print(f'HMM covariances\n {hmm_model.covars_}')
print(f'HMM transition matrix\n {hmm_model.transmat_}')
print(f'HMM initial probability\n {hmm_model.startprob_}')

In [ ]:
sp_ret = SP['return'].dropna().values.reshape(-1,1)
n_components = np.arange(1, 10)
clusters = [hmm.GaussianHMM(n_components=n, 
                            covariance_type="full").fit(sp_ret)
           for n in n_components]
plt.plot(n_components, [m.score(np.array(SP['return'].dropna())\
                                .reshape(-1,1)) for m in clusters])
plt.title('Optimum Number of States')
plt.xlabel('n_components')
plt.ylabel('Log Likelihood')

In [ ]:
hmm_model = hmm.GaussianHMM(n_components=3, 
                        covariance_type="full", 
                        random_state=123).fit(sp_ret)
hidden_states = hmm_model.predict(sp_ret)

In [ ]:
from matplotlib.dates import YearLocator, MonthLocator
from matplotlib import cm

In [ ]:
df_sp_ret = SP['return'].dropna()

hmm_model = hmm.GaussianHMM(n_components=3, 
                            covariance_type="full", 
                            random_state=123).fit(sp_ret)

hidden_states = hmm_model.predict(sp_ret)

fig, axs = plt.subplots(hmm_model.n_components, sharex=True,
                        sharey=True, figsize=(12, 9))
colors = cm.gray(np.linspace(0, 0.7, hmm_model.n_components))

for i, (ax, color) in enumerate(zip(axs, colors)):
    mask = hidden_states == i
    ax.plot_date(df_sp_ret.index.values[mask],
                 df_sp_ret.values[mask],
                 ".-", c=color)
    ax.set_title("Hidden state {}".format(i + 1), fontsize=16)
    ax.xaxis.set_minor_locator(MonthLocator())
plt.tight_layout()

In [ ]:
ret_merged.groupby('states')['return'].mean()

## Fama-French Model vs. HMM

In [ ]:
ff_merge['return'] = ff_merge['Close'].pct_change()
ff_merge.dropna(inplace=True)

In [ ]:
split = int(len(ff_merge) * 0.9)
train_ff= ff_merge.iloc[:split].dropna()
test_ff = ff_merge.iloc[split:].dropna()

In [ ]:
hmm_model = hmm.GaussianHMM(n_components=3,
                            covariance_type="full",
                            n_iter=100, init_params=" ")

In [ ]:
predictions = []

for i in range(len(test_ff)):
    hmm_model.fit(train_ff)
    adjustment = np.dot(hmm_model.transmat_, hmm_model.means_)
    predictions.append(test_ff.iloc[i] + adjustment[0])
predictions = pd.DataFrame(predictions)

In [ ]:
std_dev = predictions['return'].std()
sharpe = predictions['return'].mean() / std_dev
print('Sharpe ratio with HMM is {:.4f}'.format(sharpe))

## Fama-French Model with OLS

In [ ]:
import statsmodels.api as sm

In [ ]:
Y = train_ff['return']
X = train_ff[['Mkt-RF', 'SMB', 'HML']]

In [ ]:
model = sm.OLS(Y, X)
ff_ols = model.fit()
print(ff_ols.summary())

In [ ]:
ff_pred = ff_ols.predict(test_ff[["Mkt-RF", "SMB", "HML"]])
ff_pred.head()

In [ ]:
std_dev = ff_pred.std()
sharpe = ff_pred.mean() / std_dev
print('Sharpe ratio with FF 3 factor model is {:.4f}'.format(sharpe))

In [ ]:
split = int(len(SP['return']) * 0.9)
train_ret_SP = SP['return'].iloc[split:].dropna()
test_ret_SP = SP['return'].iloc[:split].dropna()

In [ ]:
hmm_model = hmm.GaussianHMM(n_components=3,
                            covariance_type="full",
                            n_iter=100)
hmm_model.fit(np.array(train_ret_SP).reshape(-1, 1))
hmm_predict_vol = hmm_model.predict(np.array(test_ret_SP)
                                    .reshape(-1, 1))
pd.DataFrame(hmm_predict_vol).value_counts()

## Synthetic Data Generation and Hidden Markov

In [ ]:
startprob = hmm_model.startprob_
transmat = hmm_model.transmat_
means = hmm_model.means_ 
covars = hmm_model.covars_

In [ ]:
syn_hmm = hmm.GaussianHMM(n_components=3, covariance_type="full")

In [ ]:
syn_hmm.startprob_ = startprob
syn_hmm.transmat_ = transmat 
syn_hmm.means_ = means 
syn_hmm.covars_ = covars

In [ ]:
syn_data, _ = syn_hmm.sample(n_samples=1000)

In [ ]:
plt.hist(syn_data)
plt.title('Histogram of Synthetic Data')
plt.show()

In [ ]:
plt.plot(syn_data, "--")
plt.title('Line Plot of Synthetic Data')
plt.show()